# Impact of various features on schedule adherence

In [9]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

## Data Preprocessing

In [10]:
df = pd.read_csv('../Data/All.csv')

In [11]:
df.head()

route_id route_short_name  direction_id  stop_id  \
0     11638               21             0    10573   
1     11638               21             0    13844   
2     11638               21             0     2472   
3     11638               21             0     2473   
4     11638               21             0     2474   

                        stop_name  vehicle_id  driver_id start_time  \
0  Washington St & Ashland Ave NB       13039     1980.0      11:10   
1  25th St & Greenmount Ave WB FS       13039     1980.0      11:10   
2     Washington St & Eager St NB       13039     1980.0      11:10   
3     Washington St & Chase St NB       13039     1980.0      11:10   
4   Washington St & Preston St NB       13039     1980.0      11:10   

   sched_adherence_secs           scheduled_date  ... matched_location Agency  \
0               842.555  2023-10-01 00:00:00.000  ...    WASHINGTON ST   Waze   
1               695.070  2023-10-01 00:00:00.000  ...   GREENMOUNT AVE   Waze   
2               821.367  2023-10-01 00:00:00.000  ...    WASHINGTON ST   Waze   
3               820.224  2023-10-01 00:00:00.000  ...    WASHINGTON ST   Waze   
4               832.819  2023-10-01 00:00:00.000  ...    WASHINGTON ST   Waze   

      incident_type Agency-specific Type    Agency-specific Sub Type  \
0  Disabled Vehicle        Weatherhazard  Hazard On Road Car Stopped   
1  Disabled Vehicle        Weatherhazard  Hazard On Road Car Stopped   
2  Disabled Vehicle        Weatherhazard  Hazard On Road Car Stopped   
3  Disabled Vehicle        Weatherhazard  Hazard On Road Car Stopped   
4  Disabled Vehicle        Weatherhazard  Hazard On Road Car Stopped   

                  Start time                Closed time        location  \
0  2023-10-02 11:10:33+00:00  2023-10-02 11:42:42+00:00   WASHINGTON ST   
1  2023-10-02 11:41:59+00:00  2023-10-02 12:33:41+00:00  GREENMOUNT AVE   
2  2023-10-02 11:10:33+00:00  2023-10-02 11:42:42+00:00   WASHINGTON ST   
3  2023-10-02 11:10:33+00:00  2023-10-02 11:42:42+00:00   WASHINGTON ST   
4  2023-10-02 11:10:33+00:00  2023-10-02 11:42:42+00:00   WASHINGTON ST   

  Op. Center         duration  
0    Unknown  0 days 00:32:09  
1    Unknown  0 days 00:51:42  
2    Unknown  0 days 00:32:09  
3    Unknown  0 days 00:32:09  
4    Unknown  0 days 00:32:09  

[5 rows x 23 columns]

In [12]:
df['actual_time'] = pd.to_datetime(df['actual_time'])
df['day'] = df['actual_time'].dt.dayofweek
days = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
df['day'] = df['day'].map(days)
df['month'] = df['actual_time'].dt.month_name()


In [13]:
data = df[['route_id', 'vehicle_id', 'driver_id', 'direction_id', 'sched_adherence_secs', 'incident_type', 'Agency-specific Sub Type', 'location', 'day', 'month']]

In [14]:
df.rename(columns={'incident_type': 'incident_category', 'Agency-specific Sub Type': 'incident_type'}, inplace=True)

In [15]:
df.head()

route_id route_short_name  direction_id  stop_id  \
0     11638               21             0    10573   
1     11638               21             0    13844   
2     11638               21             0     2472   
3     11638               21             0     2473   
4     11638               21             0     2474   

                        stop_name  vehicle_id  driver_id start_time  \
0  Washington St & Ashland Ave NB       13039     1980.0      11:10   
1  25th St & Greenmount Ave WB FS       13039     1980.0      11:10   
2     Washington St & Eager St NB       13039     1980.0      11:10   
3     Washington St & Chase St NB       13039     1980.0      11:10   
4   Washington St & Preston St NB       13039     1980.0      11:10   

   sched_adherence_secs           scheduled_date  ... incident_category  \
0               842.555  2023-10-01 00:00:00.000  ...  Disabled Vehicle   
1               695.070  2023-10-01 00:00:00.000  ...  Disabled Vehicle   
2               821.367  2023-10-01 00:00:00.000  ...  Disabled Vehicle   
3               820.224  2023-10-01 00:00:00.000  ...  Disabled Vehicle   
4               832.819  2023-10-01 00:00:00.000  ...  Disabled Vehicle   

  Agency-specific Type               incident_type                 Start time  \
0        Weatherhazard  Hazard On Road Car Stopped  2023-10-02 11:10:33+00:00   
1        Weatherhazard  Hazard On Road Car Stopped  2023-10-02 11:41:59+00:00   
2        Weatherhazard  Hazard On Road Car Stopped  2023-10-02 11:10:33+00:00   
3        Weatherhazard  Hazard On Road Car Stopped  2023-10-02 11:10:33+00:00   
4        Weatherhazard  Hazard On Road Car Stopped  2023-10-02 11:10:33+00:00   

                 Closed time        location Op. Center         duration  \
0  2023-10-02 11:42:42+00:00   WASHINGTON ST    Unknown  0 days 00:32:09   
1  2023-10-02 12:33:41+00:00  GREENMOUNT AVE    Unknown  0 days 00:51:42   
2  2023-10-02 11:42:42+00:00   WASHINGTON ST    Unknown  0 days 00:32:09   
3  2023-10-02 11:42:42+00:00   WASHINGTON ST    Unknown  0 days 00:32:09   
4  2023-10-02 11:42:42+00:00   WASHINGTON ST    Unknown  0 days 00:32:09   

      day    month  
0  Monday  October  
1  Monday  October  
2  Monday  October  
3  Monday  October  
4  Monday  October  

[5 rows x 25 columns]

## Analysis

### Impact of driver_id

In [16]:
driver = ols('sched_adherence_secs ~ C(driver_id)', data=df).fit()

In [17]:
sm.stats.anova_lm(driver)

df        sum_sq       mean_sq          F  PR(>F)
C(driver_id)    960.0  1.954323e+09  2.035753e+06  28.057229     0.0
Residual      25196.0  1.828150e+09  7.255715e+04        NaN     NaN

### Impact of vehicle_id

In [18]:
vehicle = ols('sched_adherence_secs ~ C(vehicle_id)', data=df).fit()

In [19]:
sm.stats.anova_lm(vehicle)

df        sum_sq       mean_sq         F  PR(>F)
C(vehicle_id)    633.0  9.215111e+08  1.455784e+06  12.98723     0.0
Residual       25523.0  2.860962e+09  1.120935e+05       NaN     NaN

### Impact of incident_category

In [20]:
incident_cat = ols('sched_adherence_secs ~ C(incident_category)', data=df).fit()

In [21]:
sm.stats.anova_lm(incident_cat)

df        sum_sq        mean_sq         F    PR(>F)
C(incident_category)      8.0  5.102482e+06  637810.299833  4.415099  0.000024
Residual              26148.0  3.777370e+09  144461.152071       NaN       NaN

### Impact of incident_type

In [22]:
incident_type = ols('sched_adherence_secs ~ C(incident_type)', data=df).fit()

In [23]:
sm.stats.anova_lm(incident_type)

df        sum_sq        mean_sq         F        PR(>F)
C(incident_type)     12.0  1.079648e+07  899707.075346  6.236469  4.105334e-11
Residual          26144.0  3.771676e+09  144265.460597       NaN           NaN

### Impact of location

In [24]:
location = ols('sched_adherence_secs ~ C(location)', data=df).fit()

In [25]:
sm.stats.anova_lm(location)

df        sum_sq        mean_sq         F        PR(>F)
C(location)     87.0  7.822253e+07  899109.523517  6.327566  1.406838e-67
Residual     26069.0  3.704250e+09  142094.064146       NaN           NaN